In [40]:
import matplotlib.pyplot as plt
import cv2
import open3d as o3d
#from open3d.web_visualizer import draw
from scipy.spatial.transform import Rotation as Rot
import numpy as np
import os
from bop_toolkit_lib import config
from bop_toolkit_lib import dataset_params
from bop_toolkit_lib import inout
from bop_toolkit_lib import misc
import copy

dataset = "Nema17_reducer_dataset"
mesh_directory = dataset+'/model_meshes'
mesh_list = {}
mesh_path_list =[]

for filename in os.listdir(mesh_directory):
    f = os.path.join(mesh_directory, filename)
    mesh_path_list.append(f)
    mesh = o3d.io.read_triangle_mesh(f)
    mesh_list[str(os.path.basename(f))]= mesh
    # checking if it is a file
    if os.path.isfile(f):
        print(f)

print(mesh_list)

assembling_order = ['Nema17.ply','sun_gear.ply','housing.ply','carrier.ply','cover.ply']

Nema17_reducer_dataset/model_meshes/housing.ply
Nema17_reducer_dataset/model_meshes/sun_gear.ply
Nema17_reducer_dataset/model_meshes/stage_2.ply
Nema17_reducer_dataset/model_meshes/planet.ply
Nema17_reducer_dataset/model_meshes/carrier.ply
Nema17_reducer_dataset/model_meshes/stage_3.ply
Nema17_reducer_dataset/model_meshes/Nema17.ply
Nema17_reducer_dataset/model_meshes/cover.ply
Nema17_reducer_dataset/model_meshes/stage_4.ply
{'housing.ply': TriangleMesh with 2314 points and 4708 triangles., 'sun_gear.ply': TriangleMesh with 660 points and 1332 triangles., 'stage_2.ply': TriangleMesh with 2344 points and 4711 triangles., 'planet.ply': TriangleMesh with 476 points and 952 triangles., 'carrier.ply': TriangleMesh with 408 points and 828 triangles., 'stage_3.ply': TriangleMesh with 4658 points and 9419 triangles., 'Nema17.ply': TriangleMesh with 1684 points and 3379 triangles., 'cover.ply': TriangleMesh with 468 points and 952 triangles., 'stage_4.ply': TriangleMesh with 5534 points and 111

In [81]:
base_frame = o3d.geometry.TriangleMesh.create_coordinate_frame(size=0.01, origin=[0, 0, 0])
transformations = {}

def adjust_mesh(name,color,R=[0,0,0],t=[0,0,0]):
    mesh = mesh_list[name]
    T_or = Rot.from_euler('zyx',R).as_matrix()
    T_pos = np.array(t).reshape(3,1)
    T= np.concatenate((T_or, T_pos), axis = 1) 
    T= np.vstack((T, [0,0,0,1] ))
    transformations[name] = T
    mesh = copy.deepcopy(mesh).transform(T)
    mesh.paint_uniform_color(color)
    
    return mesh

Nema17 = adjust_mesh(name='Nema17.ply', color=[1, 0.706, 0])
sun_gear  = adjust_mesh(name='sun_gear.ply', color=[0.5, 0.106, 0],t=[0.0,0.0,0.0585])
housing = adjust_mesh(name='housing.ply',color=[0.7, 0.806, 0],t=[0,0,0.0505])
carrier = adjust_mesh(name='carrier.ply',color=[0.9, 0.406, 0],R =[0.0, 0.0, 0],t=[0.00, 0,0.0775])
cover = adjust_mesh(name='cover.ply',color=[0.9, 0.406, 0],R =[0.0, 0.0, 0],t=[0.00, 0,0.0845])


o3d.visualization.draw_geometries([Nema17, sun_gear, housing,carrier, cover])

In [85]:
cm_1 = Nema17 + sun_gear
o3d.io.write_triangle_mesh(mesh_directory+"/stage_2.ply", cm_1)

cm_2 = cm_1 + housing
o3d.io.write_triangle_mesh(mesh_directory+"/stage_3.ply", cm_2)

cm_3 = cm_2 + carrier
o3d.io.write_triangle_mesh(mesh_directory+"/stage_4.ply", cm_3)

cm_4 = cm_3 + cover
o3d.io.write_triangle_mesh(mesh_directory+"/stage_4.ply", cm_4)

True

In [86]:
# generate assembly_poses_gt.json
import json
with open(os.path.join(dataset, "gt_assembly_poses.json"), "w") as outfile:
    assembly_poses ={}
    for i, name in enumerate(assembling_order):
        obj_data = {
        "obj_id" : i,
        "obj_name": name,    
        "R_wrt_bottom_casing" : np.ravel(np.reshape(transformations[name][0:3,0:3],(1,9))).tolist(),
        "t_wrt_bottom_casing" : (transformations[name][0:3,3]*1000).tolist()
        }
        assembly_poses[i]= obj_data
    json.dump(assembly_poses, outfile)

In [87]:
# create models info json for error calculations

models_info = {}
for obj_id, name in enumerate(assembling_order):
    
    misc.log('Processing model of object {}...'.format(name))
    
    model = inout.load_ply(mesh_directory+'/'+name)
    ref_pt = model['pts'].min(axis=0).flatten()
    size =(model['pts'].max(axis=0) - ref_pt)

    # Calculated diameter.
    diameter = misc.calc_pts_diameter(model['pts'])

    models_info[obj_id] = {
        'min_x': ref_pt[0], 'min_y': ref_pt[1], 'min_z': ref_pt[2],
        'size_x': size[0], 'size_y': size[1], 'size_z': size[2],
        'diameter': diameter
    }


# Save the calculated info about the object models.
inout.save_json(dataset+'/models_info.json', models_info)

8/18|05:48:12: Processing model of object Nema17.ply...
8/18|05:48:12: Processing model of object sun_gear.ply...
8/18|05:48:12: Processing model of object housing.ply...
8/18|05:48:12: Processing model of object carrier.ply...
8/18|05:48:12: Processing model of object cover.ply...
